In [1]:
%load_ext autoreload

%autoreload 2

In [8]:

import pandas as pd
import os
import glob
import numpy as np
import statistics as stats
import scipy
import math
import matplotlib.pyplot as plt
import random

from src.tf_tools.rnn_regression import coolRNN
from src.tf_tools.cnn_regression import originalResNet, tranferNet
from src.grammar.sentences import sentences_Cspacer

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from scipy.stats import linregress
# 3.10.4

In [14]:
def one_hot_seqs(seqs) -> np.array:
    static_1hotmap = {
        'A' : np.array([1,0,0,0]),
        'a' : np.array([1,0,0,0]),
        'C' : np.array([0,1,0,0]),
        'c' : np.array([0,1,0,0]),
        'G' : np.array([0,0,1,0]),
        'g' : np.array([0,0,1,0]),
        'T' : np.array([0,0,0,1]),
        't' : np.array([0,0,0,1]),
    }
    onehot_seqs = []
    for seq in seqs:
        onehot_seqs.append(
            [static_1hotmap[seq[i]] if seq[i] in static_1hotmap.keys() else static_1hotmap[random.choice(['A','C','G','T'])] for i in range(len(seq))]
        )
    return np.stack(onehot_seqs)

In [18]:
atac_df = pd.read_csv('Data/ATAC_164.csv', index_col=0)
atac_df.replace('Open', 1, inplace=True)
atac_df.replace('Closed', 0, inplace=True)
atac_df

/tmp/ipykernel_3190434/2180534588.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  atac_df.replace('Closed', 0, inplace=True)


,sequence,open,set,fold
label,,,,
chr1:4357588-4357888,TAGGAATTAGTTGTTTCCCGTCTCATTTGCCTGGAGATTTAAAGTG...,1,TEST,4
chr1:4360382-4360682,TTTGTGGGCATCATTTTCTGCAGAAAGTACTCAACTCACAGATACC...,1,TRAIN,7
chr1:4383541-4383841,TGATACTGGTTTATGCCCATTTTTACTTGTAGCTGCCAACACAGGT...,1,TRAIN,4
chr1:4416929-4417229,TGAGTTTCATAGCTCTCTTAATCACCTAGAGAGAAGATTCCCAAAA...,1,TRAIN,1
chr1:4785633-4785933,CAGGAAGCGGCGTAGACTCCGCGTTGCTTTACGGCTCCGCCCGGCC...,1,TRAIN,6
...,...,...,...,...
chrY:12651138-12651438,CATCAGCTGATTTTACCTCCTTTATTACCTTGAACTCCAAAGTCTG...,0,TRAIN,6
chrY:15090196-15090496,GTTGAGTGAGACGTTAGGGGATGACCATCTACATCCCACTTGGATT...,0,TRAIN,6
chrY:17416592-17416892,TGCCTGAAGGAATGGCTAATAGTCCTACCATGTGCCAGGTTTATGT...,0,TRAIN,6


In [19]:
seq_len = len(atac_df.iloc[0]['sequence'])
model = tranferNet(input_shape=(seq_len,4))
x_train = one_hot_seqs(atac_df[atac_df['set']=='TRAIN']['sequence'])
y_train = atac_df[atac_df['set'] == 'TRAIN']['open'].values

In [21]:
#earlystop_cb = keras.callbacks.EarlyStopping('val_loss', patience=15)


history = model.fit(
    x_train,
    y_train,
    batch_size=164,
    shuffle=True,
    epochs=100,
    #callbacks=[earlystop_cb],
)

Epoch 1/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 2/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 3/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 4/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 5/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 6/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 7/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 8/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 9/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 10/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 11/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 12/100
279/279 [==============================] - 3s 9ms/step - loss: 0.2480
Epoch 13/100


In [4]:
activity_df = pd.read_csv('Data/activity.csv', index_col=0)
activity_df = activity_df.set_index('label')

In [5]:
FEATURE_KEY = 'sequence'
LABEL_KEY = 'expression_log2'

In [6]:
test_df = activity_df[activity_df['test_set']]
validation_df = activity_df[activity_df['validation_set']]
train_df = activity_df[activity_df['train_set']]

x_test = one_hot_seqs(test_df[FEATURE_KEY])
x_validation = one_hot_seqs(validation_df[FEATURE_KEY])
x_train = one_hot_seqs(train_df[FEATURE_KEY])

y_test = test_df[LABEL_KEY].values
y_validation = validation_df[LABEL_KEY].values
y_train = train_df[LABEL_KEY].values


In [7]:
model = keras.Sequential()
model.add(keras.layers.Bidirectional(keras.layers.LSTM(128, dropout=0.05, recurrent_dropout=0.2), input_shape=(164,4)))
model.add(keras.layers.Dense(1))
model.compile(loss='mae',optimizer='adam')
model.summary()


2024-03-13 13:17:00.279869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79086 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:17:00.0, compute capability: 8.0
2024-03-13 13:17:01.005901: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirection  (None, 256)               136192    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 136449 (533.00 KB)
Trainable params: 136449 (533.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_validation, y_validation), verbose=1, shuffle=False)

Epoch 1/20


2024-03-13 13:17:06.760910: I external/local_xla/xla/service/service.cc:168] XLA service 0x14ca8ac8b740 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-13 13:17:06.760934: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-03-13 13:17:06.767016: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-13 13:17:06.945454: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1710353827.029099 2447084 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


675/675 [==============================] - 257s 373ms/step - loss: 2.4348 - val_loss: 2.9292
Epoch 2/20
675/675 [==============================] - 249s 369ms/step - loss: 2.4367 - val_loss: 2.9909
Epoch 3/20
675/675 [==============================] - 247s 366ms/step - loss: 2.4411 - val_loss: 2.9964
Epoch 4/20
675/675 [==============================] - 244s 362ms/step - loss: 2.4481 - val_loss: 3.0117
Epoch 5/20
675/675 [==============================] - 244s 362ms/step - loss: 2.4492 - val_loss: 3.0162
Epoch 6/20
675/675 [==============================] - 244s 362ms/step - loss: 2.4509 - val_loss: 3.0422
Epoch 7/20
675/675 [==============================] - 244s 362ms/step - loss: 2.4513 - val_loss: 3.0189
Epoch 8/20
675/675 [==============================] - 244s 362ms/step - loss: 2.4550 - val_loss: 2.9251
Epoch 9/20
675/675 [==============================] - 245s 362ms/step - loss: 2.4368 - val_loss: 3.0306
Epoch 10/20
675/675 [==============================] - 245s 363ms/step - lo

In [10]:
preds = model.predict(x_test).flatten()
linregress(preds, y_test)

322/322 [==============================] - 19s 59ms/step


LinregressResult(slope=0.20864268448805745, intercept=-0.2000370905677875, rvalue=0.005506420993006982, pvalue=0.5764064576419283, stderr=0.37347039336902854, intercept_stderr=0.6885486445903929)